In [1]:
from all_weather.portfolio import Portfolio, Allocation, Position
from all_weather.market import get_multiple_historical_data
from enum import Enum
from pydantic import BaseModel

In [2]:
class Trade(BaseModel):
    symbol: str
    action: str
    quantity: int
    price: float


class RebalancePeriod(Enum):
    DAILY = 1
    WEEKLY = 5
    MONTHLY = 21
    QUARTERLY = 63
    ANNUALLY = 252

In [7]:
portfolio = Portfolio(
    value=10000.00,
    allocations=[
        Allocation(percent=0.30, symbol="VOO", description="S&P 500"),
        Allocation(percent=0.40, symbol="VGLT", description="Long-Term Treasury"),
        Allocation(percent=0.15, symbol="VGIT", description="Intermediate-Term Treasury"),
        Allocation(percent=0.075, symbol="GLD", description="Gold"),
        Allocation(percent=0.075, symbol="GSG", description="Commodities"),
    ],
    positions=[],
)

In [5]:
symbols = [allocation.symbol for allocation in portfolio.allocations]
data = get_multiple_historical_data(symbols)
rebalance_period = RebalancePeriod.ANNUALLY  # rebalance every year
last_rebalance_date = None
trades = []

for timestamp, prices in data.iterrows():
    if not last_rebalance_date:
        # If this is the first iteration, allocate the initial portfolio
        last_rebalance_date = timestamp
        instructions = portfolio.allocate(prices=prices.to_dict())
        for instruction in instructions:
            # Initialize the position
            position = Position(symbol=instruction.symbol, quantity=instruction.quantity)
            portfolio.positions.append(position)

            # Update the portfolio value
            portfolio.value -= instruction.quantity * instruction.price

            # Record trade
            trade = Trade(
                symbol=instruction.symbol,
                action=instruction.action,
                quantity=instruction.quantity,
                price=instruction.price,
            )
            trades.append(trade)
    else:
        # Check if it's time for rebalancing
        days_since_rebalance = (timestamp - last_rebalance_date).days
        if days_since_rebalance >= rebalance_period.value:
            last_rebalance_date = timestamp
            instructions = portfolio.allocate(prices=prices.to_dict())
            for instruction in instructions:
                # Adjust positions
                for position in portfolio.positions:
                    if position.symbol == instruction.symbol:
                        position.quantity += (
                            instruction.quantity if instruction.action == "BUY" else -1 * instruction.quantity
                        )

                # Update the portfolio value
                portfolio.value -= instruction.quantity * instruction.price

                # Record trade
                trade = Trade(
                    symbol=instruction.symbol,
                    action=instruction.action,
                    quantity=instruction.quantity,
                    price=instruction.price,
                )
                trades.append(trade)

print(trades)


[Trade(symbol='VOO', action='SELL', quantity=199, price=151.81), Trade(symbol='VGLT', action='SELL', quantity=139, price=54.23), Trade(symbol='VGIT', action='SELL', quantity=68, price=52.19), Trade(symbol='GLD', action='SELL', quantity=36, price=125.72), Trade(symbol='GSG', action='BUY', quantity=124, price=32.64), Trade(symbol='VOO', action='SELL', quantity=271, price=162.93), Trade(symbol='VGLT', action='SELL', quantity=305, price=63.14), Trade(symbol='VGIT', action='SELL', quantity=192, price=54.06), Trade(symbol='GLD', action='SELL', quantity=78, price=114.29), Trade(symbol='GSG', action='SELL', quantity=279, price=19.8), Trade(symbol='VOO', action='SELL', quantity=161, price=163.58), Trade(symbol='VGLT', action='SELL', quantity=655, price=59.92), Trade(symbol='VGIT', action='SELL', quantity=246, price=53.98), Trade(symbol='GLD', action='SELL', quantity=80, price=101.92), Trade(symbol='GSG', action='SELL', quantity=630, price=15.51), Trade(symbol='VOO', action='SELL', quantity=136,